In [1]:
# standard dependencies
import os
import pandas as pd
import sys
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
# local package dependencies
sys.path.append(os.path.abspath("../../scripts/"))

# for making direct query requests to the data warehouse
from oso_db import execute_query

# Grab data for a single project

In [3]:
slug = 'gitcoin'
date = '2023-01-01'

contribs = execute_query(f"""

    SELECT 
        e.time AS time, 
        et.name AS action,
        from_artifact.name AS user, 
        to_artifact.name AS repo,
        e.amount AS amount
    FROM event e
    JOIN artifact from_artifact ON e."fromId" = from_artifact.id
    JOIN artifact to_artifact ON e."toId" = to_artifact.id
    JOIN project_artifacts_artifact paa_to ON to_artifact.id = paa_to."artifactId"
    JOIN project p ON paa_to."projectId" = p.id
    JOIN event_type et ON e."typeId" = et.id
    WHERE 
        p.slug = '{slug}'
        AND et.name IN (
            'COMMIT_CODE', 
            'PULL_REQUEST_CREATED', 
            'PULL_REQUEST_MERGED', 
            'ISSUE_CREATED', 
            'ISSUE_CLOSED'
        )
        AND e.time >= '{date}'
        
""", col_names=True)

In [28]:
df_contribs = pd.DataFrame(contribs[1:], columns=contribs[0])
df_contribs.to_csv("csv/gitcoin_contribs.csv")
df_contribs

,time,action,user,repo,amount
0,2023-12-28 17:38:39+00:00,COMMIT_CODE,larisa17,gitcoinco/passport,1.0
1,2023-12-28 22:25:11+00:00,COMMIT_CODE,digitalmnt,gitcoinco/passport-scorer,1.0
2,2023-12-28 20:16:12+00:00,COMMIT_CODE,jmcook1186,gitcoinco/passport-scorer,1.0
3,2023-12-28 23:49:56+00:00,PULL_REQUEST_CREATED,digitalmnt,gitcoinco/passport,1.0
4,2023-12-28 17:38:41+00:00,ISSUE_CLOSED,nutrina,gitcoinco/passport,1.0
...,...,...,...,...,...
15959,2023-10-03 21:55:08+00:00,PULL_REQUEST_MERGED,digitalmnt,gitcoinco/passport,1.0
15960,2023-10-02 15:23:51+00:00,PULL_REQUEST_MERGED,nutrina,gitcoinco/passport,1.0
15961,2023-10-02 15:41:19+00:00,PULL_REQUEST_MERGED,aminah-io,gitcoinco/passport,1.0
15962,2023-10-04 15:52:45+00:00,PULL_REQUEST_MERGED,lucianhymer,gitcoinco/passport,1.0


In [24]:
(
    df_contribs
    .pivot_table(
        index='user', 
        columns='action', 
        values='amount', 
        aggfunc='sum', 
        fill_value=0)
    .sort_values(
        by='COMMIT_CODE', 
        ascending=False)
    .head(15)
)

action,COMMIT_CODE,ISSUE_CLOSED,ISSUE_CREATED,PULL_REQUEST_CREATED,PULL_REQUEST_MERGED
user,,,,,
boudra,748,116,79,203,177
thelostone-mc,372,404,145,250,225
giliomeejg,347,8,9,34,32
gravityblast,337,64,111,147,131
codenamejason,327,81,56,236,182
nutrina,318,137,218,203,178
digitalmnt,278,112,23,236,212
bard,253,127,131,98,83
lucianhymer,240,153,44,235,217


# Grab stats about a collection of projects

In [5]:
collection = 'gitcoin-allo'

stats = execute_query(f"""

    SELECT 
        p.slug AS slug,
        COUNT(DISTINCT CASE WHEN a."type" = 'GIT_REPOSITORY' THEN a."id" END) AS "# GitHub Repos",
        MIN(CASE WHEN e."typeId" IN (2,4) THEN e."time" END) AS "Date First Commit",
        MAX(CASE WHEN e."typeId" = 14 THEN e."amount" END) AS "Total Stars",
        MAX(CASE WHEN e."typeId" = 22 THEN e."amount" END) AS "Total Forks",

        COUNT(DISTINCT CASE WHEN e."typeId" IN (2,4,18) THEN e."fromId" END) AS "Total Contributors",
        SUM(CASE WHEN e."typeId" = 4 THEN e."amount" END) AS "Commits Last 90 Days",        
        SUM(CASE WHEN e."typeId" = 2 THEN e."amount" END) AS "PRs Created Last 90 Days",
        SUM(CASE WHEN e."typeId" = 3 THEN e."amount" END) AS "PRs Merged Last 90 Days",        
        SUM(CASE WHEN e."typeId" = 18 THEN e."amount" END) AS "Issues Created Last 90 Days",
        SUM(CASE WHEN e."typeId" = 6 THEN e."amount" END) AS "Issues Closed Last 90 Days",
        
        COUNT(DISTINCT CASE WHEN e."typeId" IN (2,4,18) AND e."time" >= NOW() - INTERVAL '90 days' THEN e."fromId" END) AS "Contributors Last 90 Days",        
        SUM(CASE WHEN e."typeId" = 4 AND e."time" >= NOW() - INTERVAL '90 days' THEN e."amount" END) AS "Commits Last 90 Days",        
        SUM(CASE WHEN e."typeId" = 2 AND e."time" >= NOW() - INTERVAL '90 days' THEN e."amount" END) AS "PRs Created Last 90 Days",
        SUM(CASE WHEN e."typeId" = 3 AND e."time" >= NOW() - INTERVAL '90 days' THEN e."amount" END) AS "PRs Merged Last 90 Days",        
        SUM(CASE WHEN e."typeId" = 18 AND e."time" >= NOW() - INTERVAL '90 days' THEN e."amount" END) AS "Issues Created Last 90 Days",
        SUM(CASE WHEN e."typeId" = 6 AND e."time" >= NOW() - INTERVAL '90 days' THEN e."amount" END) AS "Issues Closed Last 90 Days"        
    FROM project p
    JOIN project_artifacts_artifact paa ON p."id" = paa."projectId"
    LEFT JOIN artifact a ON paa."artifactId" = a."id"
    LEFT JOIN event e ON e."toId" = paa."artifactId" 
    JOIN collection_projects_project cpp ON p."id" = cpp."projectId"
    LEFT JOIN collection c ON cpp."collectionId" = c."id"
    WHERE c.slug = '{collection}'
    GROUP BY p.slug;

""", col_names=True)

In [27]:
df_stats = pd.DataFrame(stats[1:], columns=stats[0])
df_stats.to_csv("csv/gitcoin-allo_stats.csv")
df_stats

,slug,# GitHub Repos,Date First Commit,Total Stars,Total Forks,Total Contributors,Commits Last 90 Days,PRs Created Last 90 Days,PRs Merged Last 90 Days,Issues Created Last 90 Days,Issues Closed Last 90 Days,Contributors Last 90 Days,Commits Last 90 Days,PRs Created Last 90 Days,PRs Merged Last 90 Days,Issues Created Last 90 Days,Issues Closed Last 90 Days
0,0u0-ai,2,2023-02-21 06:43:09+00:00,1.0,NaN,3,77.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
1,0x-boring-security,2,2022-03-12 23:45:46+00:00,NaN,1.0,1,2.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
2,0x-dead-list,3,2022-07-23 15:29:40+00:00,27.0,6.0,11,12729.0,4.0,2.0,1.0,1.0,1,2014.0,NaN,NaN,NaN,NaN
3,0x-echo,11,2022-05-07 03:29:48+00:00,10.0,NaN,3,905.0,3.0,3.0,14.0,NaN,0,NaN,NaN,NaN,NaN,NaN
4,0xbootleggers,8,2022-11-02 21:54:24+00:00,1.0,1.0,7,257.0,75.0,75.0,91.0,86.0,1,21.0,6.0,6.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,zkp2p,6,2023-04-01 09:01:57+00:00,197.0,21.0,10,723.0,314.0,285.0,8.0,6.0,9,307.0,141.0,132.0,5.0,5.0
399,zkpodai,2,2023-05-25 11:26:08+00:00,2.0,NaN,2,13.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN
400,zkrepl-0x-parc,1,2021-12-07 11:59:07+00:00,126.0,18.0,13,93.0,8.0,6.0,12.0,7.0,5,1.0,1.0,1.0,3.0,2.0
401,zuzalu-housing,1,2023-04-15 15:38:01+00:00,1.0,NaN,1,9.0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN


In [26]:
df_stats.sort_values(by='Contributors Last 90 Days', ascending=False).head(20)

,slug,# GitHub Repos,Date First Commit,Total Stars,Total Forks,Total Contributors,Commits Last 90 Days,PRs Created Last 90 Days,PRs Merged Last 90 Days,Issues Created Last 90 Days,Issues Closed Last 90 Days,Contributors Last 90 Days,Commits Last 90 Days,PRs Created Last 90 Days,PRs Merged Last 90 Days,Issues Created Last 90 Days,Issues Closed Last 90 Days
288,protocol-guild,278,2013-12-19 18:38:09+00:00,44645.0,19822.0,16509,230877.0,70664.0,57615.0,39651.0,33601.0,822,3188.0,1796.0,1473.0,680.0,637.0
88,defi-llama,48,2020-11-14 00:42:10+00:00,735.0,4007.0,3313,60950.0,19988.0,17899.0,330.0,282.0,608,5271.0,2323.0,2100.0,37.0,28.0
379,wagmi-dev,14,2021-11-23 03:11:29+00:00,5269.0,786.0,1262,3182.0,2690.0,2306.0,1250.0,1231.0,246,668.0,342.0,327.0,172.0,185.0
40,blockscout,19,2018-01-16 21:56:19+00:00,2976.0,2047.0,2217,21740.0,6431.0,5780.0,5511.0,5128.0,140,1190.0,578.0,535.0,360.0,284.0
121,ethereum-cat-herders,13,2015-10-27 07:44:32+00:00,12272.0,5360.0,1423,6555.0,6386.0,4777.0,2078.0,2093.0,84,115.0,122.0,79.0,51.0,63.0
47,buidlguidl,69,2018-08-14 00:15:15+00:00,8915.0,4129.0,350,6079.0,1233.0,919.0,502.0,408.0,81,248.0,224.0,180.0,102.0,97.0
205,l2beat,11,2020-05-05 20:03:59+00:00,421.0,299.0,238,3434.0,3472.0,3211.0,273.0,248.0,68,536.0,608.0,579.0,13.0,12.0
126,ethers-io,29,2016-05-03 06:14:57+00:00,7258.0,1806.0,1851,1224.0,598.0,27.0,2680.0,1999.0,67,57.0,28.0,NaN,51.0,31.0
214,lighthouse-sigp,1,2018-07-06 07:54:07+00:00,2618.0,672.0,523,5222.0,3285.0,1190.0,1745.0,1489.0,52,NaN,138.0,103.0,70.0,63.0
354,tor-project,35,2007-12-31 20:38:23+00:00,4225.0,962.0,1084,62737.0,3590.0,1949.0,852.0,713.0,45,232.0,4.0,1.0,8.0,4.0
